In [3]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [4]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type
        if d_type=='train':
            with open('./data/chat_feature_train.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='val':
            with open('./data/chat_feature_val.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='test':
            with open('./data/chat_feature_test.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        
        with open('./LSTM_feature_extractor_Audio_13_ver2/features',"rb") as f2:  
            self.audio=json.load(f2)
        with open('./video_raw_feature_sum_smooth/video_feature.pickle',"rb") as f2:  
            self.video=pickle.load(f2)
        with open('./label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.chat_result[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(23): #7 : window size
                s_window=[]
                if vframe+idx<len(self.chat_result[game_id]):
                    s_window+=((self.chat_result[game_id][vframe+idx]))#vframe의 chat
                    s_window+=(self.audio[game_id][vframe+idx])#vframe의 image
                    s_window+=self.video[game_id][vframe+idx]
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*384
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [5]:
train=Mul_data('train')
val=Mul_data('val')

print(train[100])
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844412722519367', array([[-4.23758756e-03,  8.49383231e-03,  1.44858090e-02, ...,
         2.23878518e-01, -1.45550579e-01,  3.13746005e-01],
       [ 1.31821318e-04,  3.67566757e-03,  1.84836853e-02, ...,
         2.22326949e-01, -1.46041274e-01,  3.11356783e-01],
       [ 2.78287055e-03, -2.53155967e-03,  2.11640801e-02, ...,
         2.20462933e-01, -1.46055505e-01,  3.08700144e-01],
       ...,
       [-9.56570916e-03,  2.41865851e-02,  5.35075879e-03, ...,
         2.23566726e-01, -1.45880789e-01,  3.13223928e-01],
       [-8.14033020e-03,  2.75277682e-02,  2.89613754e-03, ...,
         2.19447777e-01, -1.46146879e-01,  3.07358474e-01],
       [-7.51412148e-03,  2.00215783e-02,  8.99404753e-03, ...,
         2.22329631e-01, -1.46165714e-01,  3.11271429e-01]]), 0)


In [6]:
train[100][1].shape

(23, 384)

In [7]:
input_size=384
hidden_size=128
length=7
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        feature = out[:,-1,:]
        out = self._lin(out[:,-1,:])
        return out,feature

In [8]:
model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [11]:
weight_dir='./C_raw+A_raw+V_raw/'


In [12]:
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(200):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out,_=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out,_=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            







0
epoch 0 train_loss : 0 , val_loss : 0.39529019594192505,p 0.49197431781701445, r 0.9472406181015453, f 0.6476003622094778

1
epoch 1 train_loss : 0 , val_loss : 0.35533273220062256,p 0.4984911792014856, r 0.9481236203090507, f 0.653430701354024

2
epoch 2 train_loss : 0 , val_loss : 0.3400682210922241,p 0.5038087425290051, r 0.9490066225165563, f 0.6581949016305596

3
epoch 3 train_loss : 0 , val_loss : 0.3745499849319458,p 0.5031630740393627, r 0.9481236203090507, f 0.6574315016072247

4
epoch 4 train_loss : 0 , val_loss : 0.3295593857765198,p 0.4859541130386122, r 0.9584988962472406, f 0.6449313033791311

5
epoch 5 train_loss : 0 , val_loss : 0.35250169038772583,p 0.48574927633043863, r 0.9631346578366445, f 0.6457963291888691

6
epoch 6 train_loss : 0 , val_loss : 0.27013906836509705,p 0.5616383348949271, r 0.9262693156732892, f 0.6992750604116325

7
epoch 7 train_loss : 0 , val_loss : 0.365702360868454,p 0.4854670512536055, r 0.9660044150110375, f 0.6461901949202599

8
epoch 8 tr

epoch 66 train_loss : 0 , val_loss : 0.2729284167289734,p 0.5775956284153005, r 0.9333333333333333, f 0.7135864978902953

67


KeyboardInterrupt: 

In [13]:
test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output,_=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 1 26 0 tensor(6) tensor(5)
7 0 23 2 tensor(7) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
8 2 21 1 tensor(10) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 16 16 tensor(0) tensor(16)
13 0 15 4 tensor(13) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 7 13 7 tensor(12) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 10 22 0 tensor(10) tensor(0)
0 2 30 0 tensor(2) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 0 25 0 tensor(7) tensor(7)
12 4 10 6 tensor(16) tensor(18)
0 2 22 8 tensor(2) tensor(8)
0 16 16 0 tensor(16) tensor(0)
11 0 19 2 tens

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 3 19 3 tensor(10) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 10 22 0 tensor(10) tensor(0)
17 8 7 0 tensor(25) tensor(17)
10 0 19 3 tensor(10) tensor(13)
10 4 18 0 tensor(14) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 28 4 0 tensor(28) tensor(0)
0 15 17 0 tensor(15) tensor(0)
14 7 11 0 tensor(21) tensor(14)
3 0 21 8 tensor(3) tensor(11)
10 14 8 0 tensor(24) tensor(10)
1 0 30 1 tensor(1) tensor(2)
0 3 29 0 tensor(3) tensor(0)
0 13 19 0 tensor(13) tensor(0)
0 0 32 0 tensor(0) tensor(0)
15 4 13 0 tensor(19) tensor(15)
7 0 23 2 tensor(7) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 3 29 0 tensor(3) tensor(0)
0 9 23 0 tensor(9) tensor(0)
3 17 12 0 tensor(20) tensor(3)
32 0 0 0 tensor(32) tensor(32)
19 0 12 1 tensor(19) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 12 13 0 tensor(19) tensor(7)
27 0 4 1 tensor(27) tensor(28)
9 6 17 0 tensor(15) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 18 14 0 tensor(18) tensor(0)
11 5 16 0 tensor(16) tensor(11)
0 13 19 0 tensor(13) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 0 23 2 tensor(7) tensor(9)
5 0 23 4 tensor(5) tensor(9)
0 4 28 0 tensor(4) tensor(0)
4 5 23 0 tensor(9) tensor(4)
0 31 1 0 tensor(31) tensor(0)
10 9 13 0 tensor(19) tensor(10)
10 4 16 2 tensor(14) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
3 2 27 0 tensor(5) tensor(3)
21 6 3 2 tensor(27) tensor(23)
9 11 12 0 tensor(20) tensor(9)
0 3 29 0 tensor(3) tensor(0)
0 3 29 0 tensor(3) tensor(0)
0 8 17 7 tensor(8) tensor(7)
0 0 26 6 tensor(0) tensor(6)
12 1 19 0 tensor(13) tensor(12)
9 0 12 11 tensor(9) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 3 29 0 tensor(3) tensor(0)
0 14 18 0 tensor(14) tensor(0)
0 12 20 0 tensor(12) tensor(0)
4 3 25 0 tensor(7) tensor(4)
4 1 27 0 tensor(5) tensor(4)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 15 17 0 tensor(15) tensor(0)
0 4 28 0 tensor(4) tensor(0)
28 4 0 0 tensor(32) tensor(28)
5 13 14 0 tensor(18) tensor(5)
0 1 31 0 tensor(1) tensor(0)
23 3 6 0 tensor(26) tensor(23)
14 6 12 0 tensor(20) tensor(14)
6 3 23 0 tensor(9) tensor(6)
7 1 24 0 tensor(8) tensor(7)
29 2 1 0 tensor(31) tensor(29)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 7 25 0 tensor(7) tensor(0)
22 10 0 0 tensor(32) tensor(22)
7 9 16 0 tensor(16) tensor(7)
0 0 32 0 tensor(0) tensor(0)
6 6 20 0 tensor(12) tensor(6)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
10 0 20 2 tensor(10) tensor(12)
19 8 3 2 tensor(27) tensor(21)
8 7 17 0 tensor(15) tensor(8)
0 0 32 0 tensor

In [15]:
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
with open('./label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

102844212431058132
precision : 0.6011673151750972, recall : 0.8110236220472441, f1 : 0.6905027932960893, accuracy : 0.8875811688311688
102844341902586509
precision : 0.55, recall : 0.9565217391304348, f1 : 0.6984126984126985, accuracy : 0.9022450888681011
102844401152267937
precision : 0.652991452991453, recall : 0.9502487562189055, f1 : 0.7740628166160082, accuracy : 0.9025775447793797
102844212430927059
precision : 0.6901234567901234, recall : 0.8469696969696969, f1 : 0.7605442176870748, accuracy : 0.9074414935577176
102844412708953395
precision : 0.5578947368421052, recall : 0.7681159420289855, f1 : 0.6463414634146342, accuracy : 0.8871595330739299
102844212429944013
precision : 0.6333333333333333, recall : 0.7681940700808625, f1 : 0.6942752740560292, accuracy : 0.8773216031280547
102844341912679064
precision : 0.5857605177993528, recall : 0.8008849557522124, f1 : 0.6766355140186916, accuracy : 0.9108247422680412
102844235753749959
precision : 0.5900383141762452, recall : 0.78371501

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [11]:
x=[]

In [9]:
with open('../data/chat_feature_pred_128_train.json',"rb") as f1:  
    chat_result=json.load(f1)

In [24]:
len(chat_result['102844235753356742'])

1654

In [21]:
        with open('../data/audio_energy_2_normaized.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [25]:
audio_result['102844235753356742'][1]

4.277879204882054e-07

In [8]:
        with open('../data/audio_H.pickle',"rb") as f2:  
            image_result=pickle.load(f2)

In [10]:
len(image_result['102844235753356742'])

16571